volumePockets - machiningInOppositeDirection

# Pockets

## Pre-requisite

The present documentation uses the following concepts:
* deburring surfaces and metafaces (please see the Perimeter documentation for more details [__[html](./Perimeter.html)__][__[ipynb](./Perimeter.ipynb)__])
* GriddedBSpline (please see the GriddedBSpline documentation for more details [__[html](./GriddedBSpline.html)__][__[ipynb](./GriddedBSpline.ipynb)__])

## What

<img src="data/Pockets/drivers.png" alt="drawing" width="500"/>
<br>
 <span style="color:red">  **volumePockets**</span> is a float that represents the total volume of the pockets.<br>
 
 <span style="color:red">  **machiningInOppositeDirection**</span> is a boolean that indicates whether the part is machined in the two opposite directions.<br>

**Illustration**<br><br>
<u>Pockets</u><br><br>
Pockets are displayed in white in the part below.
<img src="data/Pockets/pockets.png" alt="drawing" width="500"/>
<br><br>
<u>Machining in opposite direction</u><br><br>
We show below two parts: one part for which <span style="color:red">  **machiningInOppositeDirection**</span> = False and another part for which <span style="color:red">  **machiningInOppositeDirection**</span> = True.<br>
* <span style="color:red">  **machiningInOppositeDirection**</span> = False
<img src="data/Pockets/machininginoppositedirection.png" alt="drawing" width="700"/>
<br>
* <span style="color:red">  **machiningInOppositeDirection**</span> = True
<img src="data/Pockets/machiningyes.png" alt="drawing" width="800"/>

## Where
The data source of the drivers is STP files.


## When
Feature extraction phase.

## How

### General strategy

### 1. To compute <span style="color:red">  **volumePockets**</span>
<span style="color:red">  **volumePockets**</span> is computed as the sum of the volumes of all the individual pockets:<br><br>
 $$volumePockets = \sum_{p} volumePocket_p$$
 <br>
The volume of individual pocket $volumePocket_p$ is computed using the area of the pocket $area_p$ and the maximum height of the pocket $height_p$:<br><br>
 $$volumePocket_p = area_p \times height_p$$
<br>

<img src="data/Pockets/volume.png" alt="drawing" width="500"/>
<br>
To get $area_p$ and $height_p$, we need to detect the pockets (see 2. To detect the pockets). Once we have the pockets, we can easily compute their areas. However, to get the heights of the pockets, we need some more work (see 3. To get the height of the pockets).

### 2. To detect the pockets
Step 1: we first identify the very specific curved faces that surround the pockets (in red in the image below).
<br>
<img src="data/Pockets/curved_faces.png" alt="drawing" width="500"/>
<br>
Step 2: We detect as pockets the faces that are surrounded by the red curved faces on more than 55% of their perimeter

### 3. To get the height of the pockets
Step 1: we first detect the walls of the pockets (in yellow in the image below).
<br>
<img src="data/Pockets/yellow.png" alt="drawing" width="500"/>
<br>
Step 2: we compute the height of each wall that surrounds the pocket, and we take the maximum height.

### 4. To extract <span style="color:red">  **machiningInOppositeDirection**</span>
A part has <span style="color:red">  **machiningInOppositeDirection**</span> = True if one of the two following conditions is verified:
* the part has pockets in opposite directions
* the part has red curved faces in opposite directions (total length of the red curved faces > 50mm for each direction)

### More detailed description
<br>
The drivers <span style="color:red">  **machiningInOppositeDirection**</span> and <span style="color:red">  **volumePockets**</span> are computed with the function <span style="color:blue">  **extract_pockets_drivers**</span>. <br>
The different steps of the method and the subfunctions associated are presented here.

<br>
**STEP 1**: <br>
**Classification of the flat / curved / deburring faces.**
* Get the flat faces (we assume that the pockets are flat)<br>
we take all faces with a maximum absolute mean curvature that is smaller than 0.01 
* Get the curved faces<br>
we take all faces with a maximum absolute mean curvature that is larger than 0.08
* Get the deburring faces<br>
the deburring faces are obtained:
 * using the function <span style="color:blue">**get_deburring_surfaces_improved**</span> if the size of the stp file is smaller than 30 000 000 bytes.
 * using the function <span style="color:blue">**get_deburring_surfaces_faces_speedup**</span> if the size of the stp file is larger than 30 000 000 bytes. In such case, the deburring faces are computed without using the metafaces. Please refer to the **Perimeter** documentation for more details [__[html](./Perimeter.html)__][__[ipynb](./Perimeter.ipynb)__].
<br><br>

**STEP 2**: <br>
**Get the direction axis of the pockets** if <br>
$$width_{bbox} > 3 \times height_{bbox}$$
<br>

This is done using the function <span style="color:blue">**get_thickness_axis_from_normals**</span>.<br>

* The flat faces that are parallel between each other are grouped. We compute the total area of all groups except those that are orthogonal to the length axis. And we take the unit normal axes $n_1$ and $n_2$ of the two groups with the biggest areas.<br><br>

* We assume that the pockets are more in the direction of $height_{bbox}$ than in that of $width_{bbox}$ if $$width_{bbox} > 3 \times height_{bbox}$$
<br>
In such case, we compute the direction $n$ of the pockets as the axis among ${n_1, n_2}$ that maximizes the absolute value of the scalar dot betzeen $n_i$ and $height_{bbox}$.
<br><br>

**STEP 3**: <br>
**Get the pockets**
<br>
We extract the pockets using the function <span style="color:blue">**get_pockets**</span>.<br>
<br>
The pockets are of different types:
* closed pockets: the face is surrounded by curved faces (obtained in step 1) on more than 90% of their perimeter.
* middle pockets: the face is surrounded by curved faces (obtained in step 1) on more than 55% of their perimeter.
* open pockets: the face is surrounded by curved faces (obtained in step 1) on more than 55% of their perimeter and the face is surrounded by curved or deburring faces (obtained in step 1) on more than 90% of their perimeter. With the thresholds chosen, an open pocket is also a middle pocket. But one can decide to modify the thresholds in future.
<br>

The function returns in outputs:
* the list of closed pockets
* the list of all pockets (closed + middle + open)
<br><br>

**STEP 4**: <br>
**Get the direction axis of the pockets** if not found previously in step 2, i.e. if:
$$width_{bbox} \leqslant 3 \times height_{bbox}$$
<br>

This is done using the function <span style="color:blue">**get_thickness_from_pockets**</span>.<br>

* We still have $n_1$ and $n_2$ computed in step 2.<br><br>

* We compute the mean unit normal axis of the closed pockets $n_p$ (the average is weighted using the area of the pockets).<br><br>

* We compute the direction $n$ of the pockets as the axis among ${n_1, n_2}$ that maximizes the absolute value of the scalar dot between $n_i$ and $n_p$.
<br><br>

**STEP 5**: <br>
**Drop the pockets and the curved faces that are not in the right direction**

* Drop the pockets and the curved faces that do have a mean unit normal that is orthogonal to the pocket direction axis (the pocket direction axis is computed in step 2 or 4)
<br><br>

**STEP 6**: <br>
**Get the walls of the shape** (in yellow in the image above)
<br>

This is done using the function <span style="color:blue">**get_list_walls**</span>.<br> 

A face is a wall if it verifies the three following conditions:
* it is a flat face
* its mean unit normal is orthogonal to the direction of the pockets
* it is located above a curved face 
<br><br>

**STEP 7**: <br>
**Get the maximum height of each pocket**
<br>

This is done using the function <span style="color:blue">**get_max_height_pocket**</span>.<br>

For each pocket:
* <u>First</u>: we get the walls associated with the pocket considered (among the walls obtained in step 6). <br>
 A wall is associated with a pocket if it is a adjacent to a curved face that is itself adjacent to the pocket considered.<br><br>
* <u>Second</u>: we compute the height of each wall associated with the pocket considered <br>
 To compute the height of a wall, we compute the scalar dot between:
 * the mean unit normal of the pocket
 * the vector formed by the two following points:
   * the center of the wall 
   * the point of the pocket (among the 25 points of the GriddedBSpline, please refer to the **GriddedBSpline** documentation for more details [__[html](./GriddedBSpline.html)__][__[ipynb](./GriddedBSpline.ipynb)__]) that is closest to the center of the wall. 
 The height is computed as twice the absolute value of this scalar dot.
* <u>Third</u>: the height considered for each pocket $height_p$ is the maximum height obtained over all walls.
<br><br>

**STEP 8**: <br>
**Get <span style="color:red">  volumePockets</span>**
<br>

<span style="color:red">  **volumePockets**</span> is computed as the sum of the volumes of all the individual pockets:<br><br>
 $$volumePockets = \sum_{p} volumePocket_p$$
 <br>
The volume of individual pocket $volumePocket_p$ is computed using the area of the pocket $area_p$ and the maximum height of the pocket $height_p$:<br><br>
 $$volumePocket_p = area_p \times height_p$$
<br>

Here is the script:
```python
volume = sum([dict_facesid2maxheight[face_sid] * dict_facesid2area[face_sid] for face_sid in list_pockets])
```

**STEP 9**: <br>
**Get <span style="color:red">  machiningInOppositeDirection</span>**
<br>

A part has <span style="color:red">  **machiningInOppositeDirection**</span> = True if one of the two following conditions is verified:
* the part has pockets in opposite directions
* the part has red curved faces in opposite directions (total length of the red curved faces > 50mm for each direction)

## Display
<br>
One can display the pockets (in white), the curved faces (in red), and the walls (in yellow) of the part by setting dis=True in the function <span style="color:blue">**extract_pockets_drivers**.
    
    
<img src="data/Pockets/yellow.png" alt="drawing" width="500"/>